<a href="https://colab.research.google.com/github/thapawan/SatelliteOverpassPredict/blob/main/Satellite_Overpass.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# load packages
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

In [ ]:
# You can rename the .csv to any desired output filename here. Default is "output_overpass_pred.csv"
# https://github.com/thapawan/dea-notebooks/blob/develop/How_to_guides/ARD_overpass_predictor.ipynb
output_path = '../content/overpass_input_US_sites.csv'

In [ ]:
# Read in base file 'overpass_input.csv'
# Edit this file with new field sites, or load your own file with field sites as needed.
# You don't need to input Path / Row or Lat / Long. The notebook only requires start times for the relevant satellites.
base_file_path = '../content/overpass_input_US_sites.csv'
overpass = pd.read_csv(base_file_path, index_col='Site', 
                       parse_dates=['landsat_8','landsat_9', 'sentinel_2a', 'sentinel_2b'])
overpass
#"NaT" values are expected - indicates a date was not entered in input file. 

,Latitude,Longitude,Path,Row,landsat_8,landsat_9,sentinel_2a,sentinel_2b
Site,,,,,,,,
Tombigbee1,32.926,-88.207,21,37,2023-06-14 16:24:00,2023-06-06 16:24:00,2023-06-06 16:45:00,2023-11-06 16:28:00
Black Warroir,33.011,-87.639,21,37,2023-06-14 16:24:00,2023-06-06 16:24:00,2023-06-06 16:45:00,2023-11-06 16:28:00
Tombigbee2,32.532,-87.853,21,37,2023-06-14 16:24:00,2023-06-06 16:24:00,2023-06-06 16:45:00,2023-11-06 16:28:00
Cahaba1,32.445,-87.180,20,38,2023-07-06 16:24:00,2023-06-15 16:24:00,2023-06-06 16:45:00,2023-11-06 16:28:00
Cahaba2,32.445,-87.180,21,37,2023-06-14 16:24:00,2023-06-06 16:24:00,2023-06-06 16:45:00,2023-11-06 16:28:00
Alabama1,32.318,-87.093,20,38,2023-07-06 16:24:00,2023-06-15 16:24:00,2023-06-06 16:45:00,2023-11-06 16:28:00
Alabama2,32.318,-87.093,21,37,2023-06-14 16:24:00,2023-06-06 16:24:00,2023-06-06 16:45:00,2023-11-06 16:28:00
Alabama3,32.318,-87.093,21,38,2023-07-06 16:24:00,2023-06-06 16:24:00,2023-06-06 16:45:00,2023-11-06 16:28:00
Alabama4,32.406,86.457,41,38,2023-06-14 16:24:00,2023-06-06 16:24:00,2023-06-06 16:45:00,2023-11-06 16:28:00


In [ ]:
# Set a list for field sites included in prediction. If adding to the field sites, please leave the 1st entry, "Mullion",
# as this is used to order secondary overpasses.
sites = list(overpass.index)
sites[0] # outputs 1st site. Should be 'Tombigbee1'

'Tombigbee1'

In [ ]:
# Set satellite timesteps for overpasses
# Depending on your application, you may want to add more accurate timesteps - but note that times used as inputs are 
# for the beginning of the acquisition, not the actual overpass time at a specific site.
l8_timestep = timedelta(days=16)
l9_timestep = timedelta (days=16)
s2a_timestep = timedelta(days=10)
s2b_timestep = timedelta(days=10)

In [ ]:
# Specify start date
l8_startdate = overpass['landsat_8'] 
l9_startdate = overpass['landsat_9'] 
sentinel2a_startdate = overpass['sentinel_2a']
sentinel2b_startdate = overpass['sentinel_2b']
sentinel2a_startdate

Site
Tombigbee1      2023-06-06 16:45:00
Black Warroir   2023-06-06 16:45:00
Tombigbee2      2023-06-06 16:45:00
Cahaba1         2023-06-06 16:45:00
Cahaba2         2023-06-06 16:45:00
Alabama1        2023-06-06 16:45:00
Alabama2        2023-06-06 16:45:00
Alabama3        2023-06-06 16:45:00
Alabama4        2023-06-06 16:45:00
Tombigbee3      2023-06-06 16:45:00
Mobile          2023-06-06 16:45:00
Name: sentinel_2a, dtype: datetime64[ns]

In [ ]:
# Landsat 8 overpass prediction for 20*(overpass frequency) - ie 20*16 = 320 days. You can change this as desired,
# to get overpass predictions for n days. n = x*(OP freq). X is arbitrarily set to 20 to give 320 days as a base example
landsat8 = list()
landsat9 = list()
for i in range(20):
    landsat8.append(l8_startdate + l8_timestep*(i))
    landsat9.append(l9_startdate + l9_timestep*(i))
landsat8 = pd.DataFrame(landsat8)
landsat9 = pd.DataFrame(landsat9)

In [ ]:
# Sentinel 2a overpass prediction for 32 * the overpass frequency, this is to give a similar total time to the L8 prediction.
# OP frequency for Sentinel = 10 days, n days = 32*10 = 320
Sentinel_2A = [] 
Sentinel_2B = [] 
for i in range(32):
    Sentinel_2A.append(sentinel2a_startdate + s2a_timestep * (i))
    Sentinel_2B.append(sentinel2b_startdate + s2b_timestep * (i))
Sentinel_2A = pd.DataFrame(Sentinel_2A)
Sentinel_2B = pd.DataFrame(Sentinel_2B)



In [ ]:
Sentinel_2A

In [ ]:
Sentinel_2B

In [ ]:
# Add satellite label to each entry in "Sat" column
Sentinel_2A['Sat'] = 'S2A'
Sentinel_2B['Sat'] = 'S2B'
landsat8['Sat'] = 'L8'
landsat9['Sat'] = 'L9'

combined = pd.concat([Sentinel_2A, Sentinel_2B, landsat8, landsat9], axis=0, ignore_index=True)
df = pd.DataFrame(combined)

In [ ]:
# Use a "time dummy" to force a re-order of data by date, to standardise across dataframes.
today = datetime.today()
timedummy = []
t0 = datetime(today.year, today.month, 1)
dummystep = timedelta(days=2)

for i in range(300):
    timedummy.append(t0 + dummystep * (i))
    
timedummy = pd.DataFrame(timedummy)
df['DateStep'] = timedummy

In [ ]:
# Create a new df for each field site of interest. To add a new field site, add in same format ie 
# "Field_Site = df[['Field_Site', 'Sat', 'DateStep']].copy()" - if you are appending to the original file simply add in "site4 = sites[3]" 
# and "Site4 = df[[(site4), 'Sat', 'DateStep']].copy()" in the appropriate section below.
site1 = sites[0]
site2 = sites[1]
site3 = sites[2]
#site4 = sites[3]

Site1 = df[[(site1), 'Sat', 'DateStep']].copy()
Site2 = df[[(site2), 'Sat', 'DateStep']].copy()
Site3 = df[[(site3), 'Sat', 'DateStep']].copy()
#Site4 = df[[(site4), 'Sat', 'DateStep']].copy()

In [ ]:
# Reorder by date for each site, include satellite tags for each date. 
# Add new lines for new sites as needed.
Site1 = (Site1.sort_values(by=[(site1), 'DateStep'])).reset_index(drop=True)
Site2 = (Site2.sort_values(by=[(site2), 'DateStep'])).reset_index(drop=True)
Site3 = (Site3.sort_values(by=[(site3), 'DateStep'])).reset_index(drop=True)
#Site4 = (Site4.sort_values(by=[(site4), 'DateStep'])).reset_index(drop=True)

In [ ]:
combined_sites = ([Site1, Site2, Site3]) ## If you have more sites, ie you have added 1 or more, add here! An example is provided below if you have a total of 5 sites.
#combined_sites = ([Site1, Site2, Site3, Site4, Site5])

merged = pd.concat(combined_sites, axis=1)
merged = merged.rename_axis("Overpass", axis="columns")
output = merged.drop(['DateStep'], axis=1)

#add timestep: (comment out / delete hash to use as applicable - must include column/site name) - ignore the error
#output['Mullion']=output['Mullion']+datetime.timedelta(hours=10)
#output['Lake_George']=output['Lake_George']+datetime.timedelta(hours=10)
#output['Narrabundah']=output['Narrabundah']+datetime.timedelta(hours=10)

output.to_csv(output_path)
output.head(20)

Overpass,Tombigbee1,Sat,Black Warroir,Sat,Tombigbee2,Sat
0,2023-06-06 16:24:00,L9,2023-06-06 16:24:00,L9,2023-06-06 16:24:00,L9
1,2023-06-06 16:45:00,S2A,2023-06-06 16:45:00,S2A,2023-06-06 16:45:00,S2A
2,2023-06-14 16:24:00,L8,2023-06-14 16:24:00,L8,2023-06-14 16:24:00,L8
3,2023-06-16 16:45:00,S2A,2023-06-16 16:45:00,S2A,2023-06-16 16:45:00,S2A
4,2023-06-22 16:24:00,L9,2023-06-22 16:24:00,L9,2023-06-22 16:24:00,L9
5,2023-06-26 16:45:00,S2A,2023-06-26 16:45:00,S2A,2023-06-26 16:45:00,S2A
6,2023-06-30 16:24:00,L8,2023-06-30 16:24:00,L8,2023-06-30 16:24:00,L8
7,2023-07-06 16:45:00,S2A,2023-07-06 16:45:00,S2A,2023-07-06 16:45:00,S2A
8,2023-07-08 16:24:00,L9,2023-07-08 16:24:00,L9,2023-07-08 16:24:00,L9
9,2023-07-16 16:24:00,L8,2023-07-16 16:24:00,L8,2023-07-16 16:24:00,L8


In [ ]:
# add landsat 8 and 9 in split window of that date it was acquired
